In [4]:
# importing all required packages & notebook extensions at the start of the notebook
import os
import pandas as pd
import qiime2 as q2
from skbio import OrdinationResults
from qiime2 import Visualization
from seaborn import scatterplot
import matplotlib.pyplot as plt
import seaborn as sns
import biom
from scipy.stats import spearmanr
from statsmodels.stats.multitest import multipletests

%matplotlib inline

In [5]:
#all variables
Data_raw='Data/raw'
Data_classified='Data/classified'
Data_diversity='Data/diversity'
Data_aroma='Data/aroma'

<div style="background-color: skyblue; padding: 10px;">
    Titles
    </div>
<div style="background-color: aliceblue; padding: 10px;">
    Results

# Prepare Data
<div style="background-color: skyblue; padding: 10px;">


Export all necessary files

In [8]:
!qiime tools export \
  --input-path $Data_classified/taxonomy.qza \
  --output-path $Data_aroma

/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Exported Data/classified/taxonomy.qza as TSVTaxonomyDirectoryFormat to directory Data/aroma/exported-taxonomy


In [6]:
taxonomy = pd.read_csv("Data/aroma/taxonomy.tsv", sep="\t")

In [13]:
!qiime tools export \
  --input-path $Data_classified/table-filtered-sourdough_only.qza \
  --output-path $Data_aroma/table-filtered-sourdough_only

/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Exported Data/classified/table-filtered-sourdough_only.qza as BIOMV210DirFmt to directory Data/aroma/table-filtered-sourdough_only


In [7]:
# Load BIOM file
table = biom.load_table('Data/aroma/table-filtered-sourdough_only/feature-table.biom')

# Convert to pandas DataFrame
table_filtered_sourdough = table.to_dataframe()

# Transpose so samples are rows, features are columns (because metadata file like this)
table_filtered_sourdough = table_filtered_sourdough.T


# Check dimensions
print(f"Shape: {table_filtered_sourdough.shape}")
print(f"First 5 samples:\n{table_filtered_sourdough.index[:5]}")
print(f"First 5 features:\n{table_filtered_sourdough.columns[:5]}")

Shape: (125, 1148)
First 5 samples:
Index(['366291_386-LP4-ITS-0386', '366291_387-LP4-ITS-0387',
       '366291_388-LP4-ITS-0388', '366291_389-LP4-ITS-0389',
       '366291_390-LP4-ITS-0390'],
      dtype='object')
First 5 features:
Index(['e275b97cfd9f38013f8b2fa887d236b5', 'cb41694c920fec98251cddcc554a3b34',
       '3a273034905d97733118d2aa8f7932c9', 'e0d94a06e608b36b4eb1f83bc0e5ce1c',
       'c12c4f383cf7839f434453e7361dbf68'],
      dtype='object')


In [ ]:
# Save to TSV
table_filtered_sourdough.to_csv('Data/aroma/table-filtered/feature-table-proper.tsv', sep='\t')

Prepare data

In [8]:
metadata = pd.read_csv("Data/raw/merged_output_usable.tsv", sep="\t") #using merged_output_usable because the spaces already substitued with _

aroma_columns = metadata.columns[-222:]

print(aroma_columns)

#so last 222 columns are aroma associated

Index(['ALCOHOLIC_D7', 'ANIMAL_FEED_D7', 'ANIMAL_STABLE_D7', 'APPLE_D7',
       'BANANA_D7', 'BEER_D7', 'BERRIES_D7', 'BREAD_D7', 'BUTTER_MILK_D7',
       'BUTYRIC_ACID_D7',
       ...
       'animal_score_D28', 'chemical_score_D28', 'body_odour_score_D28',
       'fruity_score_D28', 'maillard_score_D28', 'sour_score_D28',
       'ocean_score_D28', 'earthy_score_D28', 'fermented_dairy_score_D28',
       'nutty_score_D28'],
      dtype='object', length=222)


In [9]:
print("Now aroma df:", aroma_columns.shape, "Now metadata df:", metadata.shape)

Now aroma df: (222,) Now metadata df: (550, 304)


In [10]:
metadata_sd = metadata[metadata['sample_type'] == 'sourdough'].copy()

In [11]:
metadata_sd.head()
print("Now:", metadata_sd.shape)

Now: (125, 304)


In [12]:
# Check for missing values in aroma data
missing_aromas = metadata_sd[aroma_columns].isnull().sum()
if missing_aromas.any():
    print(f"\n⚠ Warning: Missing aroma values detected:")
    print(missing_aromas[missing_aromas > 0])


⚠ Warning: Missing aroma values detected:
PORRIDGE_D28                 21
WHOLE_GRAIN_D28              21
HAY_D28                      21
BREAD_D28                    21
CORN_D28                     21
                             ..
sour_score_D28               21
ocean_score_D28              21
earthy_score_D28             21
fermented_dairy_score_D28    21
nutty_score_D28              21
Length: 87, dtype: int64


In [11]:
metadata_sd[aroma_columns].isnull().sum(axis=1)

0       0
1       0
2       0
3       0
4      87
       ..
129     0
130     0
131     0
132    87
133     0
Length: 125, dtype: int64

In [13]:
print("Before:", metadata_sd.shape)
metadata_sd = metadata_sd.dropna(subset=aroma_columns, how='all')
print("After:", metadata_sd.shape)

Before: (125, 304)
After: (125, 304)


so there are no samples that miss all aroma analyses

In [14]:
# Extract genus from taxonomy
taxonomy['Genus'] = taxonomy['Taxon'].str.extract(r'g__([^;]+)')
taxonomy['Genus'] = taxonomy['Genus'].fillna('Unassigned')

print(f"\n✓ Extracted {taxonomy['Genus'].nunique()} unique genera")


✓ Extracted 1542 unique genera


In [15]:
# 1) Make sure taxonomy index is feature IDs
if 'Feature ID' in taxonomy.columns:
    taxonomy = taxonomy.set_index('Feature ID')

# 2) Extract genus
taxonomy['Genus'] = taxonomy['Taxon'].str.extract(r'g__([^;]+)')
taxonomy['Genus'] = taxonomy['Genus'].fillna('Unassigned')


# 3) Map ASVs to genus
feature_to_genus = taxonomy['Genus'].to_dict()

feature_table_genus = table_filtered_sourdough.copy()
feature_table_genus.columns = [
    feature_to_genus.get(col, 'Unknown') for col in feature_table_genus.columns
]

# 4) Collapse to genus
genus_table = feature_table_genus.groupby(level=0, axis=1).sum()

print("✓ Collapsed to genus level:", genus_table.shape[1], "genera")


/tmp/ipykernel_72/2546118294.py:19: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  genus_table = feature_table_genus.groupby(level=0, axis=1).sum()


✓ Collapsed to genus level: 253 genera


In [16]:
print("Genera:", genus_table.shape[1])
top_genera = genus_table.mean().sort_values(ascending=False).head(10)
print("\nTop 10 genera :")
for genus, abundance in top_genera.items():
    print(f"  {genus}: {abundance*100:.4f}%")


Genera: 253

Top 10 genera :
  Saccharomyces: 2078482.4000%
  Alternaria: 271887.2000%
  Triticum: 153444.0000%
  Unassigned: 109028.0000%
  Pichia: 39728.8000%
  Pyrenophora: 29515.2000%
  Cladosporium: 20733.6000%
  Aureobasidium: 20399.2000%
  Elymus: 16469.6000%
  Stemphylium: 7602.4000%


<div style="background-color: aliceblue; padding: 10px;">
Top 10 genera :
  Saccharomyces: 2078482.4000%  
  Alternaria: 271887.2000%  
  Triticum: 153444.0000%  
  Unassigned: 109028.0000%  
  Pichia: 39728.8000%  
  Pyrenophora: 29515.2000%  
  Cladosporium: 20733.6000%  
  Aureobasidium: 20399.2000%  
  Elymus: 16469.6000%  
  Stemphylium: 7602.4000%  

Merging metadata & genus

In [17]:
metadata = metadata.set_index('sample ID')

In [18]:
# Merge genus data with metadata
merged_data = genus_table.join(metadata, how='inner')

print(f"✓ Merged data: {merged_data.shape}")

# Define fungal columns (from genus table)
fungal_columns = genus_table.columns.tolist()

print(f"\nFinal dataset:")
print(f"  Total samples: {len(merged_data)}")
print(f"  Fungal genera: {len(fungal_columns)}")
print(f"  Aroma attributes: {len(aroma_columns)}")
print(f"  Timepoints: {sorted(merged_data['day'].unique())}")
print(f"  Backgrounds: {merged_data['background'].unique().tolist()}")

✓ Merged data: (125, 556)

Final dataset:
  Total samples: 125
  Fungal genera: 253
  Aroma attributes: 222
  Timepoints: [7.0, 14.0, 21.0]
  Backgrounds: ['non-sterile', 'sterile']


In [19]:
fungal_columns = list(fungal_columns)
aroma_columns = list(aroma_columns)

complete_cases = merged_data[fungal_columns + aroma_columns].notna().all(axis=1).sum()
print(f"  Complete cases (no missing data): {complete_cases}/{len(merged_data)}")


  Complete cases (no missing data): 104/125


## Spearman correlation
<div style="background-color: skyblue; padding: 10px;">


In [25]:
def calculate_correlations(data, fungal_columns, aroma_columns, min_samples=10):
    """Calculate Spearman correlations between fungi and aromas"""
    
    results = []
    
    for fungus in fungal_columns:
        for aroma in aroma_columns:
            # Remove samples where fungus is absent OR aroma is missing
            mask = (data[fungus] > 0) & (data[aroma].notna())
            n_samples = mask.sum()
            
            if n_samples < min_samples:
                continue
            
            x = data.loc[mask, fungus]
            y = data.loc[mask, aroma]
            
            # Skip if either variable is constant
            if x.nunique() < 2 or y.nunique() < 2:
                continue
            
            # Calculate Spearman correlation
            rho, p_value = spearmanr(x, y)
            
            results.append({
                'Fungus': fungus,
                'Aroma': aroma,
                'Spearman_rho': rho,
                'P_value': p_value,
                'N_samples': n_samples
            })
    
    results_df = pd.DataFrame(results)
    
    # Multiple testing correction (FDR)
    if len(results_df) > 0:
        results_df['FDR'] = multipletests(results_df['P_value'], method='fdr_bh')[1]
    
    return results_df


In [26]:
# Calculate correlations
cor_results = calculate_correlations(merged_data, fungal_columns, aroma_columns)

print(f"✓ Calculated {len(cor_results)} correlations")

✓ Calculated 10230 correlations


In [32]:
rho_threshold = 0.000000001
fdr_threshold = 0.00000001
sig_cors = cor_results[
    (cor_results['FDR'] < fdr_threshold) & 
    (cor_results['Spearman_rho'].abs() > rho_threshold)
].sort_values('Spearman_rho', key=abs, ascending=False)

print(f"✓ Significant correlations (FDR<{fdr_threshold}, |ρ|>{rho_threshold}): {len(sig_cors)}")

if len(sig_cors) > 0:
    print(f"\nTop 10 strongest correlations:")
    display_cols = ['Fungus', 'Aroma', 'Spearman_rho', 'FDR', 'N_samples']
    print(sig_cors.head(10)[display_cols].to_string(index=False))
    
    # Save results
    sig_cors.to_csv('significant_correlations.csv', index=False)
    print(f"\n✓ Saved to: significant_correlations.csv")
else:
    print("\n⚠ No significant correlations found!")
    print("  Consider lowering thresholds or checking data quality")

✓ Significant correlations (FDR<1e-08, |ρ|>1e-09): 0

⚠ No significant correlations found!
  Consider lowering thresholds or checking data quality


In [33]:
cor_results[['Spearman_rho', 'P_value', 'FDR']].describe()

,Spearman_rho,P_value,FDR
count,10230.000000,10230.000000,10230.000000
mean,0.008143,0.461351,0.877760
std,0.175699,0.289367,0.070993
min,-0.717127,0.000178,0.798848
25%,-0.095013,0.201637,0.798848
50%,0.015041,0.432807,0.864457
75%,0.111733,0.712533,0.948937
max,0.736213,1.000000,1.000000


<div style="background-color: aliceblue; padding: 10px;">

correlations are statistically weak also due to high number of testing

## Re-run Spearman correlation on family basis
<div style="background-color: skyblue; padding: 10px;">

to reduce reduction of statistical results through multiple testing issue

Reload taxonomy to avoid issues

In [36]:
taxonomy = pd.read_csv("Data/aroma/taxonomy.tsv", sep="\t")

In [38]:
# Extract genus from taxonomy
taxonomy['Family'] = taxonomy['Taxon'].str.extract(r'f__([^;]+)')
taxonomy['Family'] = taxonomy['Family'].fillna('Unassigned')

print(f"\n✓ Extracted {taxonomy['Family'].nunique()} unique families")


✓ Extracted 523 unique families


In [40]:
# 1) Make sure taxonomy index is feature IDs
if 'Feature ID' in taxonomy.columns:
    taxonomy = taxonomy.set_index('Feature ID')

# 2) Extract genus
taxonomy['Family'] = taxonomy['Taxon'].str.extract(r'o__([^;]+)')
taxonomy['Family'] = taxonomy['Family'].fillna('Unassigned')


# 3) Map ASVs to genus
feature_to_family = taxonomy['Family'].to_dict()

feature_table_family = table_filtered_sourdough.copy()
feature_table_family.columns = [
    feature_to_family.get(col, 'Unknown') for col in feature_table_family.columns
]

# 4) Collapse to genus
family_table = feature_table_family.groupby(level=0, axis=1).sum()

print("✓ Collapsed to family level:", family_table.shape[1], "families")


/tmp/ipykernel_72/1860116232.py:19: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  family_table = feature_table_family.groupby(level=0, axis=1).sum()


✓ Collapsed to family level: 61 families


In [44]:
print("Families:", family_table.shape[1])
top_family = family_table.mean().sort_values(ascending=False).head(10)
print("\nTop 10 families :")
for family, abundance in top_family.items():
    print(f"  {family}: {abundance*100:.4f}%")


Families: 61

Top 10 families :
  Saccharomycetales: 2124280.8000%
  Pleosporales: 424335.2000%
  Poales: 176717.6000%
  Dothideales: 21018.4000%
  Cladosporiales: 20743.2000%
  Cystofilobasidiales: 6971.2000%
  Hypocreales: 6376.8000%
  Eurotiales: 5759.2000%
  Sporidiobolales: 5386.4000%
  Filobasidiales: 4965.6000%
